In [77]:
import pandas as pd
import xml.etree.ElementTree as ET
pd.set_option('display.max_rows', 500)

In [1]:
XML_input = input() 

 E:\PMIDsMinimalExampleParsingXMLs\200.txt


In [3]:
DF_output = input()

  E:\PMIDsMinimalExampleParsingXMLs.csv


In [73]:
tree = ET.parse(XML_input)
root = tree.getroot()

In [72]:
tree = ET.parse(XML_input)
root = tree.getroot()

for node in root:
    if node.tag == "PubmedBookArticle":
        title_node = node.find('./BookDocument/ArticleTitle')
        if title_node.find("./i") != None:
            i_node = title_node.find("./i")
            print(i_node.tail.strip())
        elif title_node.find("./i") == None:
            print(title_node.text.strip())

Laing Distal Myopathy
Genetic Hearing Loss Overview
Alpha-Thalassemia
Spinocerebellar Ataxia Type 17
Related Hemochromatosis
Blepharophimosis, Ptosis, and Epicanthus Inversus Syndrome
Facioscapulohumeral Muscular Dystrophy
Familial Hemophagocytic Lymphohistiocytosis
Greig Cephalopolysyndactyly Syndrome
Zellweger Spectrum Disorder
Alpha-Thalassemia X-Linked Intellectual Disability Syndrome
Amyotrophic Lateral Sclerosis Overview
Retinoblastoma
X-Linked Agammaglobulinemia
Alkaptonuria
Craniosynostosis Syndromes Overview
Citrullinemia Type I
Fryns Syndrome
Craniometaphyseal Dysplasia, Autosomal Dominant
-Related Kyphoscoliotic Ehlers-Danlos Syndrome
Von Hippel-Lindau Syndrome
-Related Pallister-Hall Syndrome
Myotonic Dystrophy Type 2
Pendred Syndrome / Nonsyndromic Enlarged Vestibular Aqueduct
Juvenile Polyposis Syndrome
Free Sialic Acid Storage Disorders
Costeff Syndrome
Hypochondroplasia
Parkin Type of Early-Onset Parkinson Disease
Sotos Syndrome
Adenosine Deaminase Deficiency
L1 Syndrom

In [81]:
data = []
for node in root:
    article_data = {}
    if node.tag == 'PubmedArticle':
        # Type of entry
        article_data['Type'] = 'Article'

        # PMID
        pmid_node = node.find('./MedlineCitation/PMID')
        article_data['PMID'] = pmid_node.text.strip() if pmid_node is not None else None

        # Journal
        journal_node = node.find('./MedlineCitation/Article/Journal/Title')
        article_data['Journal'] = journal_node.text.strip() if journal_node is not None else None

        # Title
        title_node = node.find('./MedlineCitation/Article/ArticleTitle')
        article_data['Title'] = title_node.text.strip() if title_node is not None else None

        # Abstract
        abstract_node = node.find('./MedlineCitation/Article/Abstract')
        if abstract_node is not None:
            abstract_text = ''
            for elem in abstract_node.iter():
                if elem.text:
                    abstract_text += elem.text.strip() + ' '
            article_data['Abstract'] = abstract_text.strip()
        else:
            article_data['Abstract'] = None

        # Authors list
        authors = []
        for author in node.findall('./MedlineCitation/Article/AuthorList/Author'):
            author_info = {}
            forename_node = author.find('./ForeName')
            lastname_node = author.find('./LastName')
            if forename_node is not None and lastname_node is not None:
                author_info['Name'] = forename_node.text.strip() + ' ' + lastname_node.text.strip()
            else:
                continue

            affiliations = []
            for affiliation in author.findall('./AffiliationInfo/Affiliation'):
                affiliations.append(affiliation.text.strip())

            author_info['Affiliation'] = affiliations
            authors.append(author_info)
        article_data['Authors'] = authors

        # MeshHeadings
        mesh_headings = []
        for mesh_heading in node.findall('./MedlineCitation/MeshHeadingList/MeshHeading'):
            descriptor_name = mesh_heading.find('./DescriptorName').text.strip()
            mesh_headings.append(descriptor_name)
        article_data['MeshHeadings'] = ', '.join(mesh_headings) if mesh_headings else None

        # Chemicals
        chemicals = []
        for chemical in node.findall('./MedlineCitation/ChemicalList/Chemical'):
            chemical_name = chemical.find('./NameOfSubstance').text.strip()
            chemicals.append(chemical_name)
        article_data['Chemicals'] = ', '.join(chemicals) if chemicals else None

        # Publication types
        publication_types = []
        for publication_type in node.findall('./MedlineCitation/Article/PublicationTypeList/PublicationType'):
            publication_types.append(publication_type.text.strip())
        article_data['PublicationTypes'] = ', '.join(publication_types) if publication_types else None

        # Publication date
        pub_date_node = node.find('./MedlineCitation/Article/Journal/JournalIssue/PubDate/Year')
        if pub_date_node is not None:
            article_data['PublicationDate'] = pub_date_node.text.strip()
        else:
            article_data['PublicationDate'] = None

        # Language
        language_node = node.find('./MedlineCitation/Article/Language')
        article_data['Language'] = language_node.text.strip() if language_node is not None else None

        # Keywords
        keywords = []
        for keyword in node.findall('./MedlineCitation/KeywordList/Keyword'):
            keywords.append(keyword.text.strip())
        article_data['Keywords'] = ', '.join(keywords) if keywords else None
        
    
    elif node.tag == 'PubmedBookArticle':
        # Type of entry
        article_data['Type'] = 'Book Article'

        # PMID
        pmid_node = node.find('./BookDocument/PMID')
        article_data['PMID'] = pmid_node.text.strip() if pmid_node is not None else None

        # Book Publisher Name
        publisher_node = node.find('./BookDocument/Book/Publisher/PublisherName')
        article_data['Publisher'] = publisher_node.text.strip() if publisher_node is not None else None

        # Book title
        book_title_node = node.find('./BookDocument/Book/BookTitle')
        article_data['BookTitle'] = book_title_node.text.strip() if book_title_node is not None else None
    
        # Title
        title_node = node.find('./BookDocument/ArticleTitle')
        article_data['Title'] = title_node.text.strip() if title_node is not None else None

        title_node = node.find('./BookDocument/ArticleTitle')
        if title_node.find("./i") != None:
            i_node = title_node.find("./i")
            #article_data['Title'] = i_node.tail.strip()
            article_data["Title"] = ''.join(text.strip() for text in title_node.itertext() if text.strip())
        elif title_node.find("./i") == None and title_node != None:
            article_data['Title'] = title_node.text.strip()
        else:
            rticle_data['Title'] = None

    else:
        continue

    
    data.append(article_data)

    

    

    

    

    

    

    

In [82]:
df = pd.DataFrame(data)
df.head(100)

,Type,PMID,Publisher,BookTitle,Title,Journal,Abstract,Authors,MeshHeadings,Chemicals,PublicationTypes,PublicationDate,Language,Keywords
0,Book Article,20301606,"University of Washington, Seattle",GeneReviews,Laing Distal Myopathy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Book Article,20301607,"University of Washington, Seattle",GeneReviews,Genetic Hearing Loss Overview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Book Article,20301608,"University of Washington, Seattle",GeneReviews,Alpha-Thalassemia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Book Article,20301611,"University of Washington, Seattle",GeneReviews,Spinocerebellar Ataxia Type 17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Book Article,20301613,"University of Washington, Seattle",GeneReviews,HFE-Related Hemochromatosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Book Article,20301614,"University of Washington, Seattle",GeneReviews,"Blepharophimosis, Ptosis, and Epicanthus Inver...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Book Article,20301616,"University of Washington, Seattle",GeneReviews,Facioscapulohumeral Muscular Dystrophy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Book Article,20301617,"University of Washington, Seattle",GeneReviews,Familial Hemophagocytic Lymphohistiocytosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Book Article,20301619,"University of Washington, Seattle",GeneReviews,Greig Cephalopolysyndactyly Syndrome,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Book Article,20301621,"University of Washington, Seattle",GeneReviews,Zellweger Spectrum Disorder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
len(df)

200

In [55]:
df.loc[df["Journal"].isnull()]

,Type,PMID,Publisher,BookTitle,Journal,Title,Abstract,Authors,MeshHeadings,Chemicals,PublicationTypes,PublicationDate,Language,Keywords
0,Book Article,20301606,"University of Washington, Seattle",GeneReviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Book Article,20301607,"University of Washington, Seattle",GeneReviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Book Article,20301608,"University of Washington, Seattle",GeneReviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Book Article,20301611,"University of Washington, Seattle",GeneReviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Book Article,20301613,"University of Washington, Seattle",GeneReviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Book Article,24555207,"University of Washington, Seattle",GeneReviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,Book Article,24575447,"University of Washington, Seattle",GeneReviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,Book Article,24624459,"University of Washington, Seattle",GeneReviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,Book Article,24624460,"University of Washington, Seattle",GeneReviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv(DF_output)